In [1]:
DL = '/Users/damiansp/Downloads'

In [2]:
!rm $DL/*

rm: /Users/damiansp/Downloads/*: No such file or directory


In [3]:
from datetime import datetime
import os

import numpy as np
import pandas as pd

In [4]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'AMD', 'AMZN', 'ANET', 'APPH', 'AXON',
    'BROS', 'BYND', 'CHWY', 'COIN', 'COUR', 'CRWD', 'DDOG', 'DKNG',
    'DOCU', 'DUOL', 'EDIT', 'ETSY', 'FLGT', 'FUV', 'FVRR', 'GH', 'GMED',
    'GOOG', 'INTG', 'ISRG', 'JD', 'LSPD', 'LULU','MDB', 'MELI', 'MNST',
    'MSFT', 'MTCH', 'NVCR', 'NVDA', 'NYT', 'OKTA', 'OPEN', 'PANW',
    'PINS', 'PTON', 'QDEL', 'RBLX', 'RDFN', 'RGEN', 'ROKU', 'SE',
    'SHOP', 'SPLK', 'SQ', 'TASK', 'TDOC', 'TEAM', 'TREX', 'TTD', 'TTWO',
    'TWLO', 'TWTR', 'UPST', 'VEEV', 'WK', 'XPEV', 'ZEN', 'ZNGA']

In [6]:
!ls $DL

Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Holdings - Damian Satterthwaite-Phillips.csv


In [7]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [8]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [9]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
AMD
AMZN


### E*Trade

In [16]:
DONGMEI = False

In [17]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    #et_df.ET = et_df.ET.apply(lambda x: round(float(x)))
et_df.head()

,ET
AAPL,6123.24
ABMD,2201.71
ABNB,1300.48
AMD,101.08
AMZN,3089.21


In [18]:
et_df.loc[et_df.ET == '--']
#et_df.loc['TASK', 'ET'] = 36.41

,ET


In [19]:
check_symbols(et_df)

All clear


In [20]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,6123
ABMD,2202
ABNB,1300
AMD,101
AMZN,3089


### Fidelity

In [21]:
TODAY = datetime.now().date()
TODAY
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Apr-08-2022.csv'

In [22]:
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,ROLLOVER IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,1021.02,$1.00,$0.00,$1021.02,NaN,NaN,NaN,NaN,2.24%,NaN,NaN,Cash
1,218520300,ROLLOVER IRA,AAPL,APPLE INC,16.00,$170.09,-$2.05,$2721.44,-$32.80,-1.20%,+$297.28,+12.26%,5.97%,$2424.16,$151.51,Cash
2,218520300,ROLLOVER IRA,ABMD,ABIOMED INC COM USD0.01,1.00,$314.53,-$2.99,$314.53,-$2.99,-0.95%,+$28.67,+10.02%,0.69%,$285.86,$285.86,Cash
3,218520300,ROLLOVER IRA,ABNB,AIRBNB INC COM CL A,3.00,$162.56,-$3.35,$487.68,-$10.05,-2.02%,+$59.43,+13.87%,1.07%,$428.25,$142.75,Cash
4,218520300,ROLLOVER IRA,AMD,ADVANCED MICRO DEVICES INC,10.00,$101.00,-$2.72,$1010.00,-$27.20,-2.63%,-$15.34,-1.50%,2.22%,$1025.34,$102.53,Cash


In [23]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,ROLLOVER IRA,AAPL,2721
2,ROLLOVER IRA,ABMD,315
3,ROLLOVER IRA,ABNB,488
4,ROLLOVER IRA,AMD,1010
5,ROLLOVER IRA,ANET,1441


In [24]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['ROLLOVER IRA'].head()

,ROLLOVER IRA
Symbol,
AAPL,2721
ABMD,315
ABNB,488
AMD,1010
ANET,1441


In [25]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

ROLLOVER IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [26]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,ROLLOVER IRA,ROTH IRA,SIMPLE IRA
AAPL,6123,2721,3062,0
ABMD,2202,315,1258,0
ABNB,1300,488,975,0
AMD,101,1010,6262,0
AMZN,3089,0,3089,0


In [27]:
out.to_csv(f'{DL}/clean.csv')

In [28]:
!open $DL/clean.csv

In [33]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
mw.csv


### MarketWatch

In [10]:
!mv "$DL/Holdings - Damian Satterthwaite-Phillips.csv" "$DL/Holdings - Damian Satterthwaite-Phillips(0).csv"
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv


In [11]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv


,Value_0,Value_1,Value_2
AAPL,850,3062,2041
ABMD,1258,944,0
ABNB,975,163,0
AMD,707,303,0
AMZN,0,0,3089


In [12]:
check_symbols(mw)

All clear


In [13]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_0,Value_1,Value_2
AAPL,850,3062,2041
ABMD,1258,944,0
ABNB,975,163,0
AMD,707,303,0
AMZN,0,0,3089


In [14]:
mw.to_csv(f'{DL}/mw.csv')

In [15]:
!open $DL/mw.csv

In [15]:
!rm $DL/Holdings*.csv $DL/mw.csv
!ls $DL

In [17]:
!rm $DL/*

rm: /Users/damiansp/Downloads/*: No such file or directory


In [29]:
!ls .

03yr_pctopt.py.ipynb           format_brokerage_data.py.ipynb
05yr_pctopt.py.ipynb           individ_quantiles.csv
10yr_pctopt.py.ipynb           macd_optimizer.py.ipynb
20yr_pctopt.py.ipynb           mults.csv
30yr_pctopt.py.ipynb           old
40yr_pctopt.py.ipynb           oscillator.py.ipynb
50yr_pctopt.py.ipynb           qp50.py.ipynb
buy_multiplier.py.ipynb        scoring.csv
fair_value_individual.py.ipynb test.csv


In [36]:
!cp qp50.py.ipynb qp03.py.ipynb